In [76]:
import pandas as pd
import numpy as np
import time
import math
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)
np.random.seed(0)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device: ', device)


device:  cuda


In [77]:
# 参数
input_window = 100
output_window = 5
batch_size = 32 # batch size

# This concept is also called teacher forceing. 
# The flag decides if the loss will be calculted over all 
# or just the predicted values.
# calculate_loss_over_all_values = False
calculate_loss_over_all_values = True

In [78]:
# 位置编码
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=20000):
        super(PositionalEncoding, self).__init__()  
        # max_len参数指定模型将能够处理的最大序列长度，而d_model参数指定输入嵌入的维数
        # pe将为输入序列中的每个位置（最多max_len）提供一行，并为每个嵌入维度提供d_model列。
        # 计算位置编码的值
        pos = torch.arange(max_len).unsqueeze(1)
        i = torch.arange(d_model).unsqueeze(0)
        angle_rates = 1 / torch.pow(10000, (2 * (i // 2)) / torch.tensor(d_model).float())
        angle_rads = pos * angle_rates
        # 使用正弦和余弦函数来产生位置编码
        pe = torch.zeros((max_len, d_model))
        pe[:, 0::2] = torch.sin(angle_rads[:, 0::2])
        pe[:, 1::2] = torch.cos(angle_rads[:, 1::2])
        pe = pe.unsqueeze(0).transpose(0, 1)
#         pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):

        return x + self.pe[:x.size(0), :]

In [79]:
# 模型架构
class TransAm(nn.Module):
    def __init__(self,feature_size=7,num_layers=3,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=7, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,feature_size)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            # print('a',src.size())
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        src.size()
        # print('j',src.size(),self.src_mask.size())
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)

        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [80]:
# 划分数据窗口
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        # 数据和标签
        # 对数据进行最后output_window窗口的预测，因此用0代替
        train_seq = np.append(input_data[i:i+tw,:][:-output_window,:] , np.zeros((output_window,7)),axis=0)
        train_label = input_data[i:i+tw,:]
        #train_label = input_data[i+output_window:i+tw+output_window]
        inout_seq.append((train_seq ,train_label))
    # 转化成张量
    return torch.FloatTensor(inout_seq)

In [81]:
# 读取数据并处理
def get_data():
    
    # 导入原数据
    data = pd.read_csv('./ETTDataset/ETTh1.csv')
    c_all=pd.Series(data.columns.drop('date'))
    # 归一化
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    data_c_all = data.loc[:, c_all]
    series = data_c_all.to_numpy()
    amplitude = scaler.fit_transform(series)
    # amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    # 切分训练集和测试集
    split_train = int(amplitude.shape[0] * 0.7)
    train = amplitude[:split_train]
    test_data = amplitude[split_train:]
    split_test = int(test_data.shape[0] * 0.5)
    valid = test_data[:split_test]
    test = test_data[split_test:]
    
    # 窗口为input_window，将数据划分成(带标签的)训练和测试数据，并转化为张量
    # 转化训练数据
    train_sequence = create_inout_sequences(train,input_window)
    # 最后 output_window 为预测窗口的大小
    train_sequence = train_sequence[:-output_window]
    
    # 转化测试数据
    valid_sequence = create_inout_sequences(valid,input_window)
    valid_sequence = valid_sequence[:-output_window]
    
    # 转化测试数据
    test_sequence = create_inout_sequences(test,input_window)
    test_sequence = test_sequence[:-output_window]

    return train_sequence.to(device), valid_sequence.to(device), test_sequence.to(device),scaler

In [82]:
# 数据转化：输入标签
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]   
    # 数据切分成 输入和标签
    # 数据维度转化为[input_window,batch_size,1]
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)).squeeze() # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1)).squeeze()
    return input, target

In [83]:
train_data, valid_data, test_data, scaler = get_data()
print(train_data.size(), valid_data.size(), test_data.size())
# print(train_data.size(), val_data.size())
tr,te = get_batch(train_data, 0,batch_size)
print(tr.shape,te.shape)

torch.Size([12089, 2, 100, 7]) torch.Size([2508, 2, 100, 7]) torch.Size([2508, 2, 100, 7])
torch.Size([100, 32, 7]) torch.Size([100, 32, 7])


In [49]:
tr.shape

torch.Size([100, 32, 7])

In [50]:
tr1,te1 = get_batch(train_data, 10,batch_size)

In [51]:
tr1

tensor([[[ 1.9366e-01, -1.8023e-02,  2.7639e-01,  ..., -1.4454e-01,
           2.9635e-01, -3.9312e-02],
         [ 2.2831e-01, -1.8023e-02,  2.8643e-01,  ..., -4.3981e-02,
           2.9635e-01, -3.3721e-02],
         [ 1.8209e-01, -1.8023e-02,  2.6800e-01,  ..., -1.6973e-01,
           3.1039e-01, -7.0218e-02],
         ...,
         [ 4.3064e-01,  3.3329e-01,  5.1090e-01,  ..., -1.8233e-01,
           4.0684e-01,  4.2147e-02],
         [ 4.0462e-01,  2.8837e-01,  5.1425e-01,  ..., -1.5713e-01,
           4.2087e-01, -1.1240e-02],
         [ 4.0751e-01,  2.8837e-01,  5.0755e-01,  ..., -8.7962e-02,
           4.0684e-01,  1.7979e-01]],

        [[ 2.2831e-01, -1.8023e-02,  2.8643e-01,  ..., -4.3981e-02,
           2.9635e-01, -3.3721e-02],
         [ 1.8209e-01, -1.8023e-02,  2.6800e-01,  ..., -1.6973e-01,
           3.1039e-01, -7.0218e-02],
         [ 1.8209e-01, -3.2067e-08,  2.6635e-01,  ..., -1.1315e-01,
           3.5160e-01, -9.5494e-02],
         ...,
         [ 4.0462e-01,  2

In [92]:
tr2,te2 = get_batch(train_data, 0,1)

In [95]:
tr2.shape
tr2 = tr2.unsqueeze(1)

In [97]:
tr2.shape

torch.Size([100, 1, 7])

In [84]:
# 训练模型
def train(model, optimizer, criterion, scheduler, epoch, train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        # 数据切分成 输入数据和标签
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)
        
        # calculate_loss_over_all_values 为True时，损失将在所有输出值和目标值上计算
        # 为False时，损失仅计算 output 和 targets 的最后 output_window 个值
        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        # 执行梯度裁剪的函数,parameters参数是表示神经网络参数梯度的张量列表
        # max_norm参数是梯度的最大范数，即当参数的范数大于max_norm时，就会对梯度进行削减
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        # 计算累积的损失值
        total_loss += loss.item()
        # 计算打印日志的间隔，设置为训练集大小的5分之一
        log_interval = int(len(train_data) / batch_size / 5)
        # batch 是 log_interval 的倍数且不是第 0 个 batch时，开始打印日志
        if batch % log_interval == 0 and batch > 0:
            # 计算当前平均损失
            cur_loss = total_loss / log_interval
            # 计算从训练开始到现在的时间
            elapsed = time.time() - start_time
            # 打印日志，其中包括当前 epoch，batch，学习率，时间，损失值和困惑度
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            # 重置 total_loss，以便下一个 log_interval 计算新的平均损失值
            total_loss = 0
            start_time = time.time()

In [85]:
def plot_and_loss(eval_model, data_source,epoch,scaler):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            data = data.unsqueeze(1)
            target = target.unsqueeze(1)

            # look like the model returns static values for the output window
            output = eval_model(data)   

            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            
            test_result = torch.cat((test_result, output[-1,:].squeeze(1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1,:].squeeze(1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)
    
    print(test_result.size(),truth.size())
    test_result=scaler.inverse_transform(test_result.reshape(-1, 1)).reshape(-1)
    truth=scaler.inverse_transform(truth.reshape(-1, 1)).reshape(-1)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.axhline(y=0, color='k')
    pyplot.xlabel("Periods")
    pyplot.ylabel("Y")
    pyplot.savefig('graph/transformer-epoch%d.png'%epoch)
    pyplot.close()
    return total_loss / i


# 预测将来的时间步，steps 表示预测的步长
def predict_future(eval_model, data_source,steps,epoch,scaler):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    # 这里的 data 为真值
    _ , data = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps,1):
            input = torch.clone(data[-input_window:])
            input[-output_window:] = 0     
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    
    data=scaler.inverse_transform(data.reshape(-1, 1)).reshape(-1)
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%epoch)
    pyplot.close()
        

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    # 设置评估输入的batch大小 eval_batch_size
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data) 

            if calculate_loss_over_all_values:
                # len(data[0]) eval_batch_size的大小乘以损失
                total_loss += len(data[0])* criterion(output, targets).item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).item()   

    # 对所有验证数据输出 损失
    return total_loss / len(data_source)

In [ ]:
# 预测将来的时间步，steps 表示预测的步长
def predict_future(eval_model, data_source, steps, scaler):
    eval_model.eval() 

    # 这里的 targets 为真值, 取第一个数据[1, 2, 100, 7]进行预测将来。
    datas, targets = get_batch(data_source, 0, 1)
    data = datas.unsqueeze(1)
    with torch.no_grad():
        for i in range(0, steps, 1):
            input = torch.clone(data[-input_window:]) 
            output = eval_model(input)
            # 只将最后一行预测 进行加入
            data = torch.cat((datas, output[-1,:]))
            
    data=scaler.inverse_transform(data.reshape(-1, 1)).reshape(-1)
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-future%d.png'%steps)
    pyplot.close()

In [86]:
def plot(eval_model, data_source,epoch,scaler):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            data = data.unsqueeze(1)
            target = target.unsqueeze(1)            
            # look like the model returns static values for the output window
            output = eval_model(data)
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1,:].squeeze(1).cpu()), 0) #todo: check this. -> looks good to me
            truth = torch.cat((truth, target[-1,:].squeeze(1).cpu()), 0)
#             test_result = torch.cat((test_result, output[-1,:].cpu()), 0) #todo: check this. -> looks good to me
#             truth = torch.cat((truth, target[-1,:].cpu()), 0)
            
    #test_result = test_result.cpu().numpy()
    len(test_result)
    
    # 取[:700]的数据进行 时序预测效果的展示
    test_result_=scaler.inverse_transform(test_result[:700])
    truth_=scaler.inverse_transform(truth)
#     print(test_result.shape,truth.shape)
    for m in range(data_source.shape[-1]):
        test_result = test_result_[:,m]
        truth = truth_[:,m]
        fig = pyplot.figure(1, figsize=(20, 5))
        fig.patch.set_facecolor('xkcd:white')
        # 对测试结果 test_result [510:] 之后的数据进行预测展示
        pyplot.plot([k + 510                 for k in range(190)],test_result[510:],color="red")
        pyplot.title('Prediction uncertainty')
        pyplot.plot(truth[:700],color="black")
        pyplot.legend(["prediction", "true"], loc="upper left")
        ymin, ymax = pyplot.ylim()
        # 画出分界限
        pyplot.vlines(510, ymin, ymax, color="blue", linestyles="dashed", linewidth=2)
        pyplot.ylim(ymin, ymax)
        pyplot.xlabel("Periods")
        pyplot.ylabel("Y")
        pyplot.show()
        pyplot.close()
    return total_loss / i

In [87]:
# 获取数据和模型
# train_data, val_data,scaler = get_data()
train_data, valid_data, test_data, scaler = get_data()
model = TransAm().to(device)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_23556\1687738998.py:11: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  angle_rates = 1 / torch.pow(10000, (2 * (i // 2)) / torch.tensor(d_model).float())


In [88]:
model

TransAm(
  (pos_encoder): PositionalEncoding()
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=7, out_features=7, bias=True)
    )
    (linear1): Linear(in_features=7, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=7, bias=True)
    (norm1): LayerNorm((7,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((7,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=7, out_features=7, bias=True)
        )
        (linear1): Linear(in_features=7, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=Fa

In [89]:
# train_data, val_data,scaler = get_data()
# train_data, valid_data, test_data, scaler = get_data()
# model = TransAm().to(device)

# 一些超参数

epochs = 100 # The number of epochs
lr = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

# 初始化为正无穷大，确保第一次比较时，任何值都比 best_val_loss 更小
best_val_loss = float("inf")
best_model = None


for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    # 训练模型 损失
    train(model, optimizer, criterion, scheduler, epoch, train_data)
    
    # 验证损失
    if(epoch % 10 == 0):
        val_loss = plot(model, valid_data, epoch, scaler)
        # predict_future(model, val_data,200,epoch,scaler)
    else:
        val_loss = evaluate(model, valid_data)
        
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        # 将 best_val_loss 更新为当前的验证损失值
        best_val_loss = val_loss
        best_model = model

    scheduler.step() 

D:\Anaconda\envs\xai3\lib\site-packages\torch\optim\lr_scheduler.py:371: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |    75/  377 batches | lr 0.010000 | 34.13 ms | loss 0.09880 | ppl     1.10
| epoch   1 |   150/  377 batches | lr 0.010000 | 31.08 ms | loss 0.06616 | ppl     1.07
| epoch   1 |   225/  377 batches | lr 0.010000 | 31.24 ms | loss 0.05191 | ppl     1.05
| epoch   1 |   300/  377 batches | lr 0.010000 | 31.18 ms | loss 0.08892 | ppl     1.09
| epoch   1 |   375/  377 batches | lr 0.010000 | 30.77 ms | loss 0.07875 | ppl     1.08
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 12.67s | valid loss 0.07283 | valid ppl     1.08
-----------------------------------------------------------------------------------------


KeyboardInterrupt: 